In [4]:
# Modified from a script I got from ChatGPT
# Import necessary libraries
import pysam

# Open the SAM file
samfile = pysam.AlignmentFile('/home/rabbit/Documents/Projects/ML_gene_annot/minimap2/aln_sorted.bam', 'r')

# Open output file for intron coordinates
output_file = open('./introns.bed', 'w')

# Iterate over each read in the SAM file
for read in samfile.fetch():
    # Skip unmapped reads
    if read.is_unmapped:
        continue

    # Parse the CIGAR string to extract intron regions
    cigar_tuples = read.cigartuples
    introns = []
    current_position = read.reference_start
    for cigar in cigar_tuples:
        cigar_type, cigar_length = cigar
        if cigar_type == 3:  # 3 represents the CIGAR operation 'N' for intron
            intron_start = current_position
            intron_end = current_position + cigar_length
            introns.append((intron_start, intron_end))
        if cigar_type in [0, 2, 3, 7, 8]:  # Operations that consume reference positions
            current_position += cigar_length

    # Get the strand information from the SAM flag
    if read.is_reverse:
        strand = '-'
    else:
        strand = '+'

    # Write intron coordinates with strand to the output file
    for intron_start, intron_end in introns:
        output_file.write(f"{read.reference_name}\t{intron_start}\t{intron_end}\t.\t.\t{strand}\n")

# Close the files
samfile.close()
output_file.close()